<a href="https://colab.research.google.com/github/Firojpaudel/Machine-Learning-Notes/blob/main/Chapter1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Classification : Cats and Dogs <br>
<i>>> The first course project and its breakdown.</i>

In [2]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42, label_func=is_cat, item_tfms=Resize(224)
)

learn = vision_learner(dls, resnet34, metrics=error_rate)
# learn.fine_tune(1)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 73.8MB/s]


In [5]:
import ipywidgets as widgets

uploader = widgets.FileUpload()


In [6]:
display(uploader)

FileUpload(value={}, description='Upload')

The image inserted here was:
<div style="text-align: center;">
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/71/2010-kodiak-bear-1.jpg/1200px-2010-kodiak-bear-1.jpg" alt="A bear" width="300" height="200">
</div>

In [7]:
from fastai.vision.all import *
from PIL import Image

img = PILImage.create(uploader.data[0])
is_cat,_,probs = learn.predict(img)
print(f"Is this a cat?: {is_cat}.")
print(f"Probability it's a cat: {probs[1].item():.6f}")

Is this a cat?: False.
Probability it's a cat: 0.462523


## Breakdown of how this code works:

In [6]:
# The first line:
from fastai.vision.all import *

This line imports al the fastai.vision library. This gives us all the functions and classes we will need to create a wide variety of computer vision models.

In [7]:
#The second line:
path  = untar_data(URLs.PETS)/'images'

This line downloads a standard dataset from the fast.ai datasets collection and returns to the variabke named as "path"

In [8]:
#The function part:
def is_cat(x): return x[0].isupper()

so here, we define a funtion is_cat that labels cats based on filename rule provided by dataset creators.

In [9]:
#then we prepare the dataloader using the fast.ai functions as:
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct= 0.2, seed=42, label_func=is_cat, item_tfms=Resize(224)
)

So here, the dataloader we created works as following:
The ```.from_name_func``` is the method from fast.ai library which we use genreally for ImageDataLoaders. Then, ```path``` is the path we specified above. Similarly, ```get_image_files()``` is the function that retrieves all images files from the specified path. Then comes ```valid_pct``` which is simply setting aside 20% of the data for validation. Likewise, ```seed``` is used to ensure reproducibility by setting a random seed. And finally, ```item_tmfs = Resize(224)``` applies a transformation to resize all images to 224×224 pixels.

<details>
  <summary><b>Why 224 pixels? </b><i> (click to view)</i></summary>
- It is the standard size for historical reasons. Old pretrained models require this size exactly.
But, we can pass pretty much anything.
</details>


<details>
  <summary><b>Why validation set? And how is effects in accuracy if we use trainset instead?</b></summary>
- Having a validation set is crtitical. Because if we train the large enough model for long amount of time, the model will eventually memorize the label every time in our dataset. And will completely overfit. So we test the accuracy in unseen data by splitting the dataset to train and validation sets.
</details>

<i> Slight note: fast.ai has by default ```valid_pct``` set to 0.2 so that even if we forget to add it in, fast.ai does that for us.</i>

In [10]:
#learning part:
learn = cnn_learner(dls, resnet34, metrics= error_rate)

/usr/local/lib/python3.10/dist-packages/fastai/vision/learner.py:303: UserWarning: `cnn_learner` has been renamed to `vision_learner` -- please update your code
  warn("`cnn_learner` has been renamed to `vision_learner` -- please update your code")
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 142MB/s]


So, this line of code simply is for deep learning. Here, we have the ```cnn_learner()``` funtion that is the function in fast.ai for cnn architecture.

Then, we pass the dataloader we created earlier as its first parameter. Then the specific cnn architecture we would prefer. We're using ```resnet32``` here. And, the metric which we are calculating.

<i> We are calculating error_rate in this exercise.</i>

<details>
  <summary>
    <b>What is Resnet34? And what is 34 here?</b>
  </summary>
- Resnet is the more refined CNN architecure that we use for image datasets. The "34" here stands for layers. <br>
- Other variants include 18, 50, 101, 152.<br>
- Models using more layers take longer to train and are more prone to overfitting.
</details>

In [11]:
# Then last line:
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.169304,0.021718,0.006089,00:50


epoch,train_loss,valid_loss,error_rate,time
0,0.063684,0.016792,0.005413,00:53


What's it doing? Just fitting into the ```learn``` model we just created. Or lets say finetuning. Here 1 stands for number of epochs which in this case is 1.

<details>
  <summary>
    <b>But why call fine_tune, and not fit? </b>
  </summary>
- fastai does have the <code>fit</code> method which looks at images in the training set multiple times, each time updating the parameters to make predictions closer and closer to target labels.<br>
- But in this case, we have started with a pretrained model, and we dont want to throw away its capabilities. And this method is called as <b><i>finetuning</i></b>
</details>

<details>
  <summary>
    <b>What is a metric? And how does it differ from loss?</b>
  </summary>
- Loss is a measure of how well the model's predictions match the actual target values. Major purpose of loss is to guide the training process.
<br>
<br>
- Metrics are used to evaluate the performance of model in a way that is more interpretable and relevant to specific problem. Used to assess the quality of model's predictions and are often used for model evaluation and comparison.
</details>

<details>
  <summary>
    <b>What is the "head" of the model?</b>
  </summary>
- It refers to the final layer or the layers that produce the output predictions. It's the part of model that directly interacts with the task-specific data, such as classification labels or regression values.
</details>

<Details>
  <summary>
    <b>
      What kinds of features do the early layers of a CNN find? How about the later layers?
    </b>
  </summary>
- <b>Early layers:</b> These layers typically detect the patterns such as edges, lines and textures. They focus on low-level features and are the fundamental building blocks of the image.
<br>
- <b>Later Layers:</b> These layers combine the basic features detected by the earlier layers to identify more complex patterns and structures.  
</Details>